In [1]:
import pandas as pd
import yfinance as yf
#import pandas_ta as ta
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## Aquisição dos dados

In [24]:
input_path = "C:/Users/wbaldin/Desktop/fase04/Obesity.csv"

df = pd.read_csv(input_path)
df = df.rename_axis('ds').sort_index()
df.tail()

,Gender,Age,Height,Weight,family_history,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,Obesity
ds,,,,,,,,,,,,,,,,,
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III
2110,Female,23.664709,1.738836,133.472641,yes,yes,3.0,3.0,Sometimes,no,2.863513,no,1.026452,0.714137,Sometimes,Public_Transportation,Obesity_Type_III


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          2111 non-null   object 
 1   Age             2111 non-null   float64
 2   Height          2111 non-null   float64
 3   Weight          2111 non-null   float64
 4   family_history  2111 non-null   object 
 5   FAVC            2111 non-null   object 
 6   FCVC            2111 non-null   float64
 7   NCP             2111 non-null   float64
 8   CAEC            2111 non-null   object 
 9   SMOKE           2111 non-null   object 
 10  CH2O            2111 non-null   float64
 11  SCC             2111 non-null   object 
 12  FAF             2111 non-null   float64
 13  TUE             2111 non-null   float64
 14  CALC            2111 non-null   object 
 15  MTRANS          2111 non-null   object 
 16  Obesity         2111 non-null   object 
dtypes: float64(8), object(9)
memory u

In [26]:
unico = df["Obesity"].unique()
print(unico)

['Normal_Weight' 'Overweight_Level_I' 'Overweight_Level_II'
 'Obesity_Type_I' 'Insufficient_Weight' 'Obesity_Type_II'
 'Obesity_Type_III']


Criando modelo de classificação

In [27]:
x = df.drop("Obesity", axis=1)
y = df["Obesity"]

In [28]:
x.head()

,Gender,Age,Height,Weight,family_history,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
ds,,,,,,,,,,,,,,,,
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation


In [29]:
y

ds
0             Normal_Weight
1             Normal_Weight
2             Normal_Weight
3        Overweight_Level_I
4       Overweight_Level_II
               ...         
2106       Obesity_Type_III
2107       Obesity_Type_III
2108       Obesity_Type_III
2109       Obesity_Type_III
2110       Obesity_Type_III
Name: Obesity, Length: 2111, dtype: object

Tratando variáveis categóricas

In [30]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [31]:
colunas = x.columns


In [ ]:
one_hot = make_column_transformer(
    (OneHotEncoder(drop="if_binary"), ["Gender", "CAEC", "MTRANS,"]
     ),
    remainder="passthrough",
     sparse_threshold=0)

In [34]:
print(repr(one_hot))


ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                  transformers=[('onehotencoder',
                                 OneHotEncoder(drop='if_binary'),
                                 ['Gender', 'CAEC', 'MTRANS', 'family_history',
                                  'FAVC', 'SMOKE', 'SCC', 'CALC'])])


In [35]:
x = one_hot.fit_transform(x)


In [36]:
one_hot.get_feature_names_out(colunas)

array(['onehotencoder__Gender_Male', 'onehotencoder__CAEC_Always',
       'onehotencoder__CAEC_Frequently', 'onehotencoder__CAEC_Sometimes',
       'onehotencoder__CAEC_no', 'onehotencoder__MTRANS_Automobile',
       'onehotencoder__MTRANS_Bike', 'onehotencoder__MTRANS_Motorbike',
       'onehotencoder__MTRANS_Public_Transportation',
       'onehotencoder__MTRANS_Walking',
       'onehotencoder__family_history_yes', 'onehotencoder__FAVC_yes',
       'onehotencoder__SMOKE_yes', 'onehotencoder__SCC_yes',
       'onehotencoder__CALC_Always', 'onehotencoder__CALC_Frequently',
       'onehotencoder__CALC_Sometimes', 'onehotencoder__CALC_no',
       'remainder__Age', 'remainder__Height', 'remainder__Weight',
       'remainder__FCVC', 'remainder__NCP', 'remainder__CH2O',
       'remainder__FAF', 'remainder__TUE'], dtype=object)

In [38]:
pd.DataFrame(x, columns = one_hot.get_feature_names_out(colunas))

,onehotencoder__Gender_Male,onehotencoder__CAEC_Always,onehotencoder__CAEC_Frequently,onehotencoder__CAEC_Sometimes,onehotencoder__CAEC_no,onehotencoder__MTRANS_Automobile,onehotencoder__MTRANS_Bike,onehotencoder__MTRANS_Motorbike,onehotencoder__MTRANS_Public_Transportation,onehotencoder__MTRANS_Walking,...,onehotencoder__CALC_Sometimes,onehotencoder__CALC_no,remainder__Age,remainder__Height,remainder__Weight,remainder__FCVC,remainder__NCP,remainder__CH2O,remainder__FAF,remainder__TUE
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,21.000000,1.620000,64.000000,2.0,3.0,2.000000,0.000000,1.000000
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,21.000000,1.520000,56.000000,3.0,3.0,3.000000,3.000000,0.000000
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,23.000000,1.800000,77.000000,2.0,3.0,2.000000,2.000000,1.000000
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,27.000000,1.800000,87.000000,3.0,3.0,2.000000,2.000000,0.000000
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,22.000000,1.780000,89.800000,2.0,1.0,2.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,20.976842,1.710730,131.408528,3.0,3.0,1.728139,1.676269,0.906247
2107,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,21.982942,1.748584,133.742943,3.0,3.0,2.005130,1.341390,0.599270
2108,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,22.524036,1.752206,133.689352,3.0,3.0,2.054193,1.414209,0.646288
2109,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,24.361936,1.739450,133.346641,3.0,3.0,2.852339,1.139107,0.586035


Tratando a variavel Target

In [39]:
from sklearn.preprocessing import LabelEncoder

# Definir ordem das categorias
ordem_categorias = [
    'Insufficient_Weight',
    'Normal_Weight',
    'Overweight_Level_I',
    'Overweight_Level_II',
    'Obesity_Type_I',
    'Obesity_Type_II',
    'Obesity_Type_III'
]

# Converter para categoria ordenada
y = pd.Categorical(y, categories=ordem_categorias, ordered=True)

# Usar LabelEncoder para converter em números (0 a 6)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [40]:
print(y[:50])  # últimos 50

[1 1 1 5 6 1 1 1 1 1 2 6 1 2 1 1 6 2 6 5 6 2 1 2 1 1 1 1 1 1 5 6 1 6 1 6 1
 1 1 6 5 1 1 1 1 6 1 1 1 1]


Fazendo o treinamento


In [41]:
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split( x, y, stratify = y, random_state=42) 
                                                        
#stratify = y (manter a proporção das classes alvo)


dummy modelo

In [42]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier()
dummy.fit(x_treino, y_treino)

,strategy,'prior'
,random_state,None
,constant,None


In [43]:
dummy = DummyClassifier()
dummy.fit(x_treino, y_treino)

dummy.score(x_teste, y_teste)

0.16666666666666666

In [44]:
from sklearn.tree import DecisionTreeClassifier
arvore = DecisionTreeClassifier(random_state =42)

In [45]:

print(x_treino[:50])  # últimos 50

[[1.       0.       0.       ... 2.       0.       1.541493]
 [0.       0.       0.       ... 2.597746 1.922234 1.376124]
 [1.       0.       0.       ... 2.35752  0.291309 0.897942]
 ...
 [0.       0.       1.       ... 2.       0.       1.      ]
 [1.       0.       0.       ... 1.259454 0.935217 0.9976  ]
 [1.       0.       0.       ... 2.       1.       1.      ]]


In [46]:
arvore = DecisionTreeClassifier(random_state =42)
arvore.fit(x_treino, y_treino)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


após ter transformado as colunas e ter aplicado o treino, vamos predizer os resultados


In [47]:
y_pred = arvore.predict(x_teste)

#printando a acuracia da predição 

print("Acurácia:", accuracy_score(y_teste, y_pred))


Acurácia: 0.9071969696969697


fazendo a  matriz de confusão para o modelo de arvore de decisão

In [48]:
#fazendo a  matriz de confusão para o modelo de arvore de decisão
cm = confusion_matrix(y_teste, y_pred)
print("Matriz de Confusão:\n", cm)

Matriz de Confusão:
 [[60  8  0  0  0  0  0]
 [ 1 62  0  0  0  9  0]
 [ 0  1 82  1  0  1  3]
 [ 0  0  4 70  0  0  0]
 [ 0  0  1  0 80  0  0]
 [ 0 11  0  0  0 61  1]
 [ 0  0  5  0  0  3 64]]


In [52]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

In [ ]:
nome_colunas = ['Gender', 'Weight', 'family_history', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'Obesity']